# 資料庫查詢

In [5]:
# 導入庫
import os
import time
import pymysql
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.parse import quote
from datetime import datetime
from collections import defaultdict

# 載入環境變數
load_dotenv()
EMAIL = os.getenv("COUPANG_EMAIL")
PASSWORD = os.getenv("COUPANG_PASSWORD")

# 資料庫
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT", 3306)),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME")
}

# 自訂函數：從資料庫提取所有不重複的搜尋關鍵字
def fetch_distinct_keywords(db_config):
    conn = pymysql.connect(
        host=db_config["host"],
        port=db_config["port"],
        user=db_config["user"],
        password=db_config["password"],
        database=db_config["database"],
        charset="utf8mb4",
        cursorclass=pymysql.cursors.DictCursor
    )
    with conn:
        with conn.cursor() as cursor:
            cursor.execute("""
                SELECT DISTINCT search_keyword FROM coupang_products
                WHERE search_keyword IS NOT NULL AND search_keyword <> ''
            """)
            return [row["search_keyword"] for row in cursor.fetchall()]

# 自訂函數：查詢分析函數，依每個關鍵字列出價格異動商品
def analyze_keyword(keyword, db_config):
    # 連線資料庫
    conn = pymysql.connect(
        host=db_config["host"],
        port=db_config["port"],
        user=db_config["user"],
        password=db_config["password"],
        database=db_config["database"],
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )
    sql = """
        SELECT title, price, timestamp, source_type
        FROM coupang_products
        WHERE search_keyword LIKE %s AND source_type = 'login'
        ORDER BY title, timestamp
    """

    with conn:
        with conn.cursor() as cursor:
            cursor.execute(sql, (f"%{keyword}%",))
            rows = cursor.fetchall()

    # 去除 title、timestamp 重複
    grouped = defaultdict(list)
    seen = set()
    for row in rows:
        key = (row['title'], row['timestamp'])
        if key in seen:
            continue
        seen.add(key)
        grouped[row['title']].append((row['price'], row['timestamp'], row['source_type'] or "guest"))

    # 拆分：有價格異動的與只有單一價格的
    multi_price_grouped = {}
    single_price_grouped = {}
    for title, entries in grouped.items():
        prices = set(price for price, _, _ in entries)
        if len(prices) >= 2:
            multi_price_grouped[title] = entries
        elif len(prices) == 1:
            single_price_grouped[title] = entries

    # 輸出
    print(f"\n📌 關鍵字：『{keyword}』")
    # 查無資料
    if not multi_price_grouped and not single_price_grouped:
        print("⚠️ 無相關資料")
        return
    # 有異動資料
    if multi_price_grouped:
        print("\n📈 發現價格異動：")
        for title, records in multi_price_grouped.items():
            print(f"🔹 標題：{title}")
            for price, timestamp, source in sorted(records, key=lambda x: x[1]):
                print(
                    f"   - 價格：{price}"
                    f"（時間：{timestamp}，來源：{source}）"
                )
            print("-" * 60)
    # 僅有單一價格
    if single_price_grouped:
        print("\n📌 僅出現單一價格：")
        for title, records in single_price_grouped.items():
            price, timestamp, source = records[0]
            print(
                f"🔸 標題：{title}\n   - 價格：{price}"
                f"（時間：{timestamp}，來源：{source}）"
            )
        print("-" * 60)

# 主程式
if __name__ == "__main__":
    keywords = fetch_distinct_keywords(DB_CONFIG)
    if not keywords:
        print("⚠️ 查無歷史關鍵字")
        exit()

    print(
        f"🔍 共發現 {len(keywords)} 組歷史關鍵字，開始分析..."
    )

    for idx, kw in enumerate(keywords, 1):
        print(f"\n[{idx}] 分析關鍵字：{kw}")
        analyze_keyword(kw, DB_CONFIG)


🔍 共發現 13 組歷史關鍵字，開始分析...

[1] 分析關鍵字：DORO CAT 豆乳貓

📌 關鍵字：『DORO CAT 豆乳貓』

📈 發現價格異動：
🔹 標題：DORO CAT 豆乳貓 活性碳與小蘇打雙重除臭顆粒低過敏極細豆腐貓砂, 無香, 7L, 6包
   - 價格：$775（時間：2025-05-19 14:58:53，來源：login）
   - 價格：$872（時間：2025-05-19 19:49:56，來源：login）
------------------------------------------------------------
🔹 標題：DORO CAT 豆乳貓 豆腐貓砂, 抹茶, 7L, 1袋
   - 價格：$142（時間：2025-05-19 14:58:53，來源：login）
   - 價格：$164（時間：2025-05-19 19:49:56，來源：login）
------------------------------------------------------------
🔹 標題：DORO CAT 豆乳貓 豆腐貓砂, 抹茶, 7L, 6袋
   - 價格：$775（時間：2025-05-19 14:58:53，來源：login）
   - 價格：$872（時間：2025-05-19 19:49:56，來源：login）
------------------------------------------------------------

📌 僅出現單一價格：
🔸 標題：DORO CAT 豆乳貓 活性碳與小蘇打雙重除臭顆粒低過敏極細豆腐貓砂, 無香, 7L, 1包
   - 價格：$165（時間：2025-05-19 14:58:53，來源：login）
🔸 標題：DORO CAT 豆乳貓 活性碳與小蘇打雙重除臭顆粒低過敏極細豆腐貓砂, 無香, 7L, 2包
   - 價格：$330（時間：2025-05-19 14:58:53，來源：login）
🔸 標題：DORO CAT 豆乳貓 活性碳與小蘇打雙重除臭顆粒低過敏極細豆腐貓砂, 無香, 7L, 4包
   - 價格：$632（時間：2025-05-19 14:58:53，來源：login）
🔸 標題：DORO CAT 豆乳貓 混合砂, 原味, 7